# 1. import data & libraries

In [233]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
#from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from __future__ import division
from itertools import chain
from nltk.probability import *
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
import re

In [17]:
# load train and test data
train = pandas.read_csv("/Users/yezhenwang/Documents/Monash/Monash semester 2 2018/Quora/data/train.csv")
test = pandas.read_csv("/Users/yezhenwang/Documents/Monash/Monash semester 2 2018/Quora/data/test.csv")

In [9]:
ls

README.md          Tokenize.ipynb     all/               quora_demo1.ipynb


In [12]:
train = pd.read_csv("all/train.csv")
test = pd.read_csv("all/test.csv")

# 2. Randomly pick some data to chek

In [222]:
# Randomly pick rows from dataset
# target = -1, don't care about it
# target = 1, insincere
# target = 0, normal label
# default display count is 10.
def pickRawDataByTarget(target, num=10):
    random_list = []
    if target == -1:
        for i in range(num):
            random_list.append(np.random.random_integers(len(train)+1))
        for row in random_list:
            print("Q:%d \n%s || Target: %s"%(row, train.iloc[row,1], train.iloc[row,2]))
    else:
        # Create a correspond target index list at first
        targetList = (train[train['target'] == target].index).tolist()
        for i in range(num):
            random_list.append(np.random.random_integers(len(targetList)))
        for index in random_list:
            print("Q:%d \n%s || Target: %s"%(targetList[index], train.iloc[targetList[index],1], train.iloc[targetList[index],2]))

In [221]:
pickRawDataByTarget(-1)

Q:1104471 
Do you have that one skill or a set of few skills that you know will never desert you when you need it? How did you develop it? And how did you discover that those are the skills you wanted to develop? || Target: 0
Q:203810 
If a student worker on the F1 visa program works for 21 hours in a certain week just once by mistake I that a very serious offence? What imapct will it have on other H1B application in th future || Target: 0
Q:1085416 
Should I let my son watch porn and masturbate with his twin sister? || Target: 1
Q:861616 
What do you, as an atheist, think about Tim O'Neill and atheists who believe in the existence of Jesus and deny the history of Christian atrocities? || Target: 0
Q:420564 
What are the most effective techniques and procedures to perform a tubal ligation surgery? || Target: 0
Q:166543 
Whatre some of the best books to read for Java, application and system design? || Target: 0
Q:1088805 
When will Hindus rise up and invade the world? || Target: 1
Q:105

/Users/solar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. Please call randint(1, 1306123 + 1) instead
  if __name__ == '__main__':


In [200]:
#train_token_tag_df.iloc[819941,2]

## Lines might need caution
1. No.338434 12th -> twelveth<br>
2. No.1286381 Military Service: Capital case in the middle of sentence, check it whether could be bigram.
3. No.968909 man-bun with "-"
4. No.421977 22-Jun-2018, 'JJ' POS (I think it's Wrong?)
5. No.819941 Rice production/distribution/processing, dealing with '/'
    <br>Q: Stemming & N-gram order? (ref: https://stackoverflow.com/questions/47219389/compute-word-n-grams-on-original-text-or-after-lemma-stemming-process)
6. No.1298566 mis-spelling.
7. No.713198 ...Math question.


# 3. Related Functions for pre-processing: case_Normaliztion, spec_Replacement, etc.

In [77]:
def case_Normalize(line):
    '''
        'Originially used in my FIT5196 assignment, manully picked some end-tokens of a sentence'
        Write a regex to match all words with a first capital letter in front of line or with some marks 
        ahead except the marks which would occur within a sentecne like [)(&,;-]. I think it not performed perfect but 
        it is better than sent_detector.tokenize()
        
        'Here is a caution in 5196, I'm not sure whether it happened here, but it seems no side-effect'
        Caution: as data always contain many space tokens, I replace them with single space at first.
    '''
    casePattern = re.compile(r"(^[A-Z]\w*|[^\w\&,\)\(;-]\s[A-Z][a-z]+)")
    # Some files have too many space tokens among two words in a sentecne
    line = re.sub( '\s+', ' ', line).strip()
    # string.lower() used to normalise the capital letter.
    return casePattern.sub(lambda x: x.group(0).lower(), line)

In [158]:
def spec_Replacement(line):
    '''
        A regex for split specific punctuation like '/'
        1. replace '/' with ' or ' for furhter tokenize.
        2. replace '%' with ' percentage'
        3. replace '$' with 'dollar '
    '''
    line = re.sub( '\/', ' or ', line).strip()
    line = re.sub( '\$', 'dollars ', line).strip()
    line = re.sub( '\%', ' percentage', line).strip()
    

    return line

In [224]:
# Add any other functions here if need.
# Add more functions inside #3 part. Then add them into pre-processing function
def dealing_Non_Alphanumeric(line):
    pass

In [223]:
# This cell used for simple test
line1 = train.iloc[819941,1]
line1 = re.sub( '\s+', ' ', line1).strip()
#line1
line1 = case_Normalize(line1)
spec_Replacement(line1)

'how can I invest in Rice production or distribution or processing in Nigeria?'

# 4. Pre-processing

In [160]:
# Add related pre-processing function here
def pre_process(line):
    line = case_Normalize(line)
    line = spec_Replacement(line)
    
    return line

In [230]:
# Define funtion to get a stopword list
# Default using nltk corpus.
# Might add alterations.
def get_StopWords():
    return set(stopwords.words('english'))

In [81]:
# build question lists for train and test
train_question_list = train["question_text"].tolist()
test_question_list = test["question_text"].tolist()

## Q: Stemming or Lemmatization

> In FIT5196, we are required to do stemming.<br>
> https://stackoverflow.com/questions/49354665/should-i-perform-both-lemmatization-and-stemming

In [ ]:
# tokenize questions in train
train_token_list = []
for i in train_question_list:
    line = pre_process(i)
    unigram_tokens = nltk.word_tokenize(line)

    stemmer = PorterStemmer()
    stemed_tokens = list(map(lambda x: stemmer.stem(x) if not x[0].isupper() else x, unigram_tokens))
    stopped_tokens = [w for w in stemed_tokens if w not in get_StopWords()]
    
    # final_tokens remove any tokens have a tiny length (< 3)
    final_tokens = list(filter(lambda x: len(x) >= 3, stopped_tokens))
    train_token_list.append(final_tokens)
    
# tokenize questions in test
test_token_list = []
for i in test_question_list:
    line = pre_process(i)
    unigram_tokens = nltk.word_tokenize(line)

    stemmer = PorterStemmer()
    stemed_tokens = list(map(lambda x: stemmer.stem(x) if not x[0].isupper() else x, unigram_tokens))
    stopped_tokens = [w for w in stemed_tokens if w not in get_StopWords()]
    
    # final_tokens remove any tokens have a tiny length (< 3)
    final_tokens = list(filter(lambda x: len(x) >= 3, stopped_tokens))
    train_token_list.append(final_tokens)
    test_token_list.append(nltk.word_tokenize(line))

In [18]:
# tag train_token_list
train_token_tag = []
for i in train_token_list:
    train_token_tag.append(nltk.pos_tag(i))

test_token_tag = []
# tag test_token_list
for i in test_token_list:
    test_token_tag.append(nltk.pos_tag(i))

In [20]:
# finalize train and test tokenization
train_token_tag_df = pd.DataFrame({"question_text": train_question_list,
                                  "tokened_question": train_token_list,
                                  "tagged_question": train_token_tag})

test_token_tag_df = pd.DataFrame({"question_text": test_question_list,
                                  "tokened_question": test_token_list,
                                  "tagged_question": test_token_tag})

In [ ]:
#train_token_tag_df.iloc[103501,2]

In [21]:
train_token_tag_df.head()

,question_text,tokened_question,tagged_question
0,How did Quebec nationalists see their province...,"[How, did, Quebec, nationalists, see, their, p...","[(How, WRB), (did, VBD), (Quebec, NNP), (natio..."
1,"Do you have an adopted dog, how would you enco...","[Do, you, have, an, adopted, dog, ,, how, woul...","[(Do, VBP), (you, PRP), (have, VB), (an, DT), ..."
2,Why does velocity affect time? Does velocity a...,"[Why, does, velocity, affect, time, ?, Does, v...","[(Why, WRB), (does, VBZ), (velocity, NN), (aff..."
3,How did Otto von Guericke used the Magdeburg h...,"[How, did, Otto, von, Guericke, used, the, Mag...","[(How, WRB), (did, VBD), (Otto, NNP), (von, NN..."
4,Can I convert montra helicon D to a mountain b...,"[Can, I, convert, montra, helicon, D, to, a, m...","[(Can, MD), (I, PRP), (convert, VB), (montra, ..."


In [48]:
#len(train_token_tag_df) == len(train)

True

In [22]:
test_token_tag_df.head()

,question_text,tokened_question,tagged_question
0,My voice range is A2-C5. My chest voice goes u...,"[My, voice, range, is, A2-C5, ., My, chest, vo...","[(My, PRP$), (voice, NN), (range, NN), (is, VB..."
1,How much does a tutor earn in Bangalore?,"[How, much, does, a, tutor, earn, in, Bangalor...","[(How, WRB), (much, JJ), (does, VBZ), (a, DT),..."
2,What are the best made pocket knives under $20...,"[What, are, the, best, made, pocket, knives, u...","[(What, WP), (are, VBP), (the, DT), (best, JJS..."
3,Why would they add a hypothetical scenario tha...,"[Why, would, they, add, a, hypothetical, scena...","[(Why, WRB), (would, MD), (they, PRP), (add, V..."
4,What is the dresscode for Techmahindra freshers?,"[What, is, the, dresscode, for, Techmahindra, ...","[(What, WP), (is, VBZ), (the, DT), (dresscode,..."


In [49]:
test_token_tag_df.tagged_question[2]

KeyError: 1278565

In [38]:
test_token_tag_df.question_text[2]

'What are the best made pocket knives under $200-300?'